In [2]:
from __future__ import division

import numpy as np
import pandas as pd

from scipy import stats
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW, ztest

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
data_exp = np.array( [1 if i < 10 else 0 for i in range(34)] )
data_ctrl = np.array( [1 if i < 4 else 0 for i in range(16)] )

In [4]:
print('Mean experimental value: %.4f' % data_exp.mean())
print('Mean control value: %.4f' % data_ctrl.mean())

Mean experimental value: 0.2941
Mean control value: 0.2500


In [5]:
conf_interval_banner_exp = proportion_confint(np.sum(data_exp), len(data_exp), method = 'wilson')
conf_interval_banner_ctrl = proportion_confint(np.sum(data_ctrl), len(data_ctrl), method = 'wilson')

In [6]:
print('95%% confidence interval for exp group: [%f, %f]' % conf_interval_banner_exp)
print('95%% confidence interval for a ctrl group: [%f, %f]' % conf_interval_banner_ctrl)

95% confidence interval for exp group: [0.168346, 0.461689]
95% confidence interval for a ctrl group: [0.101821, 0.494983]


In [7]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [8]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [9]:
def proportions_diff_z_test(z_stat, alternative='two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)


In [10]:
print('95%% confidence interval for a difference: [%.4f, %.4f]' 
      % proportions_diff_confint_ind(data_exp, data_ctrl))

95% confidence interval for a difference: [-0.2176, 0.3058]


In [11]:
print('p-value: %.4f' % proportions_diff_z_test(proportions_diff_z_stat_ind(data_exp, data_ctrl), 'greater'))

p-value: 0.3729


In [13]:
data = pd.read_table('banknotes.txt')

In [19]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [20]:
y = data['real']
X = data.drop(['real'], axis = 1)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=1)

In [26]:
X_train_1 = X_train[['X1', 'X2', 'X3']]

log_clf_1 = LogisticRegression().fit(X_train_1, y_train)

/Users/alina/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
X_train_2 = X_train[['X4', 'X5', 'X6']]

log_clf_2 = LogisticRegression().fit(X_train_2, y_train)

/Users/alina/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
X_test_1 = X_test[['X1', 'X2', 'X3']]

pred_1 = log_clf_1.predict(X_test_1)

In [29]:
X_test_2 = X_test[['X4', 'X5', 'X6']]

pred_2 = log_clf_2.predict(X_test_2)

In [30]:
accuracy_1 = accuracy_score(y_test, pred_1)
accuracy_2 = accuracy_score(y_test, pred_2)

In [37]:
print('Errors (1, 2):', (1 - accuracy_1, 1 - accuracy_2))

Errors (1, 2): (0.19999999999999996, 0.020000000000000018)


In [44]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [42]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [45]:
err_1 = np.array([1 if y_test.values[i] == pred_1[i] else 0 for i in range(len(pred_1))])
err_2 = np.array([1 if y_test.values[i] == pred_2[i] else 0 for i in range(len(pred_2))])

print('95%% confidence interval for a difference between predictions: [%.4f, %.4f]' %
      proportions_diff_confint_rel(err_1, err_2))

95% confidence interval for a difference between predictions: [-0.3001, -0.0599]


In [46]:
print('p-value: %f' % proportions_diff_z_test(proportions_diff_z_stat_rel(err_1, err_2)))

p-value: 0.003297


In [47]:
avg_mark = 541.4
avg_mark_data = np.ones(100) * avg_mark
exp_mark = 525.
st_dev = 100.

In [48]:
def z_test(mean_val, exp_val, st_dev, num):
    standard_error = st_dev / np.sqrt(num)
    return (mean_val - exp_val) / standard_error

In [49]:
def p_val_greater(z_stat):
    return 1 - stats.norm.cdf(z_stat)

In [50]:
z_stat = z_test(avg_mark, exp_mark, st_dev, (len(avg_mark_data)))
z_stat

1.6399999999999977